In [2]:
import pandas as pd

In [65]:
data =pd.read_csv('./data/book2.csv')

In [5]:
data.head()

,SemesterYear,SemesterMonth,StudentID,ClassStart,ClassEnd,ClassPointTotal
0,2019,2,ID-02259,2/13/2019,2/27/2019,322.0
1,2019,3,ID-02259,2/13/2019,2/27/2019,322.0
2,2019,3,ID-02259,2/27/2019,3/28/2019,1023.0
3,2019,4,ID-03045,2/7/2019,3/12/2019,672.0
4,2019,4,ID-01099,3/20/2019,4/19/2019,1073.0


In [6]:
data['ClassStart'] = pd.to_datetime(data['ClassStart'])
data['ClassEnd'] = pd.to_datetime(data['ClassEnd'])

In [7]:
data.head()

,SemesterYear,SemesterMonth,StudentID,ClassStart,ClassEnd,ClassPointTotal
0,2019,2,ID-02259,2019-02-13,2019-02-27,322.0
1,2019,3,ID-02259,2019-02-13,2019-02-27,322.0
2,2019,3,ID-02259,2019-02-27,2019-03-28,1023.0
3,2019,4,ID-03045,2019-02-07,2019-03-12,672.0
4,2019,4,ID-01099,2019-03-20,2019-04-19,1073.0


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221344 entries, 0 to 221343
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   SemesterYear     221344 non-null  int64         
 1   SemesterMonth    221344 non-null  int64         
 2   StudentID        221344 non-null  object        
 3   ClassStart       221344 non-null  datetime64[ns]
 4   ClassEnd         221344 non-null  datetime64[ns]
 5   ClassPointTotal  221344 non-null  float64       
dtypes: datetime64[ns](2), float64(1), int64(2), object(1)
memory usage: 10.1+ MB


In [9]:
data['StudentID'].value_counts()
# 15495 student 

ID-17480         102
ID-02727          69
ID-02831          48
ID-12895          45
ID-01632          43
                ... 
ID-21811           1
ID-22278           1
ID-23456           1
ID-25866           1
ID-40165           1
Name: StudentID, Length: 15495, dtype: int64

In [10]:
def get_gap_records(student_id):
    stdent_data = data[data['StudentID']==student_id]
    stdent_data.sort_values(by=[ 'ClassStart','ClassEnd'])
    unique_records = stdent_data.sort_values(by=[ 'ClassStart','ClassEnd']).drop_duplicates(subset=[ 'ClassStart','ClassEnd'], keep='last')
    unique_records['gap_time'] = unique_records['ClassStart'] - unique_records['ClassEnd'].shift(1)
    return list(unique_records[unique_records['gap_time']>pd.Timedelta(days=1)].index)


In [35]:


from itertools import chain
def get_overlap_records_by_student_id(student_id):
    stdent_data = data[data['StudentID']==student_id]
    unique_records = stdent_data.sort_values(by=[ 'ClassStart']).drop_duplicates(subset=[ 'ClassStart','ClassEnd'], keep='last')
    unique_records['original_index'] = unique_records.index
    unique_records =unique_records.reset_index()
    # for each record see if there is a record that that have start data < enddate of the current record
    def check_over_lap (index,end):
        return [idx for idx in unique_records[(unique_records['ClassStart']<end)&(unique_records.index>index)]['original_index'].values]
    result =[check_over_lap(index, end_date) for index, end_date in zip(unique_records.index, unique_records['ClassEnd'])]
    return list(chain.from_iterable(result))


In [ ]:
dict_logs = {}
for idx,student_id in enumerate(data['StudentID'].unique()):
    print(idx/15495)
    over_lap = get_overlap_records_by_student_id(student_id)
    gap = get_gap_records(student_id)
    dict_logs[student_id]={'over_lap':over_lap,'gap':gap}
    print('\n')

In [50]:
data['original_id'] = data.index

In [52]:
# save data
data.to_csv('./data/data_indexed.csv')

In [53]:
logs = pd.DataFrame(columns=['StudentID','over_lap','gap'])
logs['StudentID'] = data['StudentID'].unique()
logs['over_lap'] = data['StudentID'].apply(lambda x: dict_logs[x]['over_lap'])
logs['gap'] = data['StudentID'].apply(lambda x: dict_logs[x]['gap'])

In [54]:
#save logs
logs.to_csv('./data/logs.csv')

In [55]:
# question 3 : 
data_without_doubles = data.drop_duplicates(subset=['StudentID', 'ClassStart','ClassEnd'], keep='last')

In [69]:
dublicates_ids  = list(set(list(data.index)) - set(list(data_without_doubles.index)))
dublicate_records = data[data.index.isin(dublicates_ids)]

In [70]:
dublicate_records.to_csv('./data/dublicate_records.csv')